In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
#from tqdm.auto import tqdm
from tqdm import tqdm
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in tqdm(os.listdir(maindir + '\\testing_modules')):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)
plt.close('All')

100%|██████████| 23/23 [00:12<00:00,  1.80it/s]


In [2]:
%matplotlib qt
from matplotlib.widgets import Slider

In [3]:
base = 'D:\\Musterman_postdoc\\20240223_Musterman\\'
xrf_dir = f'{base}xrf_maps\\'

In [4]:
base_scan = 153000
FXI_first = [102, 104, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 143, 145]
FXI_second = [157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215]
LiNbO = [253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297]
BNL_inner = [485, 487, 489, 491, 493, 495, 502, 504, 506, 508, 510]

In [184]:
energy_list = []
data_list = []

for scan in base_scan + np.array(LiNbO):
    with h5py.File(f'{xrf_dir}scan2D_{scan}_xs_sum8ch.h5', 'r') as f:

        xrf_fit_names = [d.decode('utf-8') for d in f['xrfmap/detsum/xrf_fit_name'][:]]
        xrf_fit = f['xrfmap/detsum/xrf_fit'][:]

        i0 = f['xrfmap/scalers/val'][..., 0]
        xrf_fit = np.concatenate((xrf_fit, np.expand_dims(i0, axis=0)), axis=0)
        xrf_fit_names.append('i0')

        xrf_dict = dict(zip(xrf_fit_names, xrf_fit))
        energy = f['xrfmap/scan_metadata'].attrs['instrument_mono_incident_energy']

    energy_list.append(energy)
    data_list.append(xrf_dict)

#sort_data = sorted(zip(energy_list, data_list), key=lambda pair: pair[0])
#energy_list = [y for (y, x) in sort_data]
#data_list = [x for (y, x) in sort_data]

In [47]:
element_key = 'Sb_L'
fig = plt.figure(figsize=(5, 5), dpi=200)
ax = fig.add_axes([0.1, 0.1, 0.5, 0.8])

element_img = ax.imshow(data_list[0][element_key] / data_list[0]['i0'])

slider_ax = fig.add_axes([0.7, 0.1, 0.03, 0.8])
energy_slider = Slider(
    ax=slider_ax,
    label='Index',
    valmin=0,
    valmax=len(energy_list) - 1,
    valinit=0,
    orientation='vertical'
)

# The function to be called anytime a slider's value changes
def update(val):
    global element_img
    val_ind = int(np.round(val, 0))
    element_img.remove()
    element_img = ax.imshow(data_list[val_ind][element_key] / data_list[val_ind]['i0'])
    fig.canvas.draw_idle()
    
energy_slider.on_changed(update)

plt.show()

In [186]:
save_dir = f'{base}plots//'

element_key = 'Er_L'
for ind in range(len(data_list)):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

    scaler = data_list[ind]['i0'].copy()
    scaler[scaler < 1000] = np.mean(scaler)
    scaler[scaler > 1e6] = np.median(scaler)

    ax.imshow(data_list[ind][element_key] / scaler, vmin=0.2, vmax=0.35)

    plt.savefig(f'{save_dir}LiNbO_{ind}.png', transparent=True)
    plt.close()

In [9]:
element_key = 'Pt_L'
fig = plt.figure(figsize=(5, 5), dpi=200)
ax = fig.add_axes([0.1, 0.1, 0.5, 0.8])

element_img = ax.imshow(data_list[0][element_key] / data_list[0]['i0'])

slider_ax = fig.add_axes([0.7, 0.1, 0.03, 0.8])
energy_slider = Slider(
    ax=slider_ax,
    label='Energy [keV]',
    valmin=energy_list[0],
    valmax=energy_list[-1],
    valinit=energy_list[0],
    orientation='vertical'
)

# The function to be called anytime a slider's value changes
def update(val):
    global element_img
    diff_arr = np.array(energy_list) - val
    val_ind = np.argmin(np.abs(diff_arr))
    element_img.remove()
    element_img = ax.imshow(data_list[val_ind][element_key] / data_list[val_ind]['i0'])
    fig.canvas.draw_idle()
    
energy_slider.on_changed(update)

plt.show()

In [187]:
ind = 13
element = 'Sb_L'
ref_image = data_list[0][element] / data_list[0]['i0']
moving_image = data_list[ind][element] / data_list[ind]['i0']

In [188]:
from skimage.registration import phase_cross_correlation

shift, error, phasediff = phase_cross_correlation(ref_image, moving_image, upsample_factor=1000, normalization=None)
print(shift)

[-6.618 -0.104]


In [189]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.imshow(ref_image, extent=[0, ref_image.shape[1], 0, ref_image.shape[0]])

ax.set_xlim(0, ref_image.shape[1])
ax.set_ylim(0, ref_image.shape[0])

plt.show()

In [190]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.imshow(moving_image, extent=[0 + shift[1],
                                ref_image.shape[1] + shift[1],
                                0 - shift[0],
                                ref_image.shape[0] - shift[0]])

ax.set_xlim(0, ref_image.shape[1])
ax.set_ylim(0, ref_image.shape[0])

plt.show()

In [50]:
from skimage.transform import warp_polar, rotate

In [163]:
def rotation_scale_translation(reference_image, moving_image, rotation_upsample=1000, shift_upsample=1000, bandpass=(0, 1), fix_rotation=False):
    from skimage.filters import window, difference_of_gaussians
    from scipy.fft import fft2, fftshift
    from skimage.transform import warp_polar, rotate
    from skimage.registration import phase_cross_correlation
    
    if not fix_rotation:
        # Image bandpass
        reference_image = difference_of_gaussians(reference_image, *bandpass)
        moving_image = difference_of_gaussians(moving_image, *bandpass)

        # Window images
        ref_wimage = reference_image * window('hann', reference_image.shape)
        mov_wimage = moving_image * window('hann', reference_image.shape)

        # work with shifted FFT magnitudes
        ref_fs = np.abs(fftshift(fft2(ref_wimage)))
        mov_fs = np.abs(fftshift(fft2(mov_wimage)))

        # Create log-polar transform
        shape = ref_fs.shape
        radius = shape[0] // 8  # only take lower frequencies
        warped_ref_fs = warp_polar(ref_fs, radius=radius, output_shape=shape,
                                scaling='log', order=0)
        warped_mov_fs = warp_polar(mov_fs, radius=radius, output_shape=shape,
                                scaling='log', order=0)
        
        # Register shift in polar space with half of FFT
        warped_ref_fs = warped_ref_fs[:shape[0] // 2, :]  # only use half of FFT
        warped_mov_fs = warped_mov_fs[:shape[0] // 2, :]
        polar_shift, polar_error, polar_phasediff = phase_cross_correlation(warped_ref_fs,
                                                                            warped_mov_fs,
                                                                            upsample_factor=rotation_upsample,
                                                                            normalization=None)
        
        # Convert to useful parameters
        shiftr, shiftc = polar_shift[:2]
        angle = (360 / shape[0]) * shiftr
        klog = shape[1] / np.log(radius)
        scale = np.exp(shiftc / klog)

        # Correct image and register translation
        # adj_image = rescale(moving_image, 2 - scale) # this seem wrong
        adj_image = rotate(moving_image, -angle)
        # adj_image = rescale
    else:
        angle = 0
        scale = 0
        adj_image = moving_image

    trans_shift, trans_error, trans_phasediff = phase_cross_correlation(ref_image,
                                                                        adj_image,
                                                                        upsample_factor=shift_upsample,
                                                                        normalization=None)
    
    return angle, scale, trans_shift

In [108]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

index = 3

scaler = data_list[index]['i0'].copy()
scaler[scaler < 1000] = np.mean(scaler)

moving_image = data_list[index][element] / scaler

ax.imshow(moving_image)

plt.show()

In [178]:
element = 'Er_L'
ref_image = data_list[3][element] / data_list[3]['i0']
ref_image[ref_image < 0.88*np.max(ref_image)] = 0

angle_list, scale_list, shift_list = [], [], []

for index in range(len(data_list)):

    scaler = data_list[index]['i0'].copy()
    scaler[scaler < 1000] = np.mean(scaler)
    scaler[scaler > 1e6] = np.median(scaler)

    moving_image = data_list[index][element] / scaler

    moving_image[moving_image < 0.89*np.max(moving_image)] = 0

    if index == 15:
        #break
        pass

    angle, scale, trans_shift = rotation_scale_translation(ref_image, moving_image, bandpass=(0, 1), fix_rotation=True)
    print(f'{angle:.3f}', f'{scale:.2f}', trans_shift)
    angle_list.append(angle)
    scale_list.append(scale_list)
    shift_list.append(trans_shift)

0.000 0.00 [34.907 73.139]
0.000 0.00 [-43.144 -80.854]
0.000 0.00 [ 13.999 -44.922]
0.000 0.00 [0.005 0.004]
0.000 0.00 [-0.102  0.694]
0.000 0.00 [0.145 0.336]
0.000 0.00 [-0.026  0.567]
0.000 0.00 [0.651 0.16 ]
0.000 0.00 [ 0.827 -0.038]
0.000 0.00 [ 1.204 -0.505]
0.000 0.00 [1.162 0.012]
0.000 0.00 [1.663 0.518]
0.000 0.00 [ 1.693 -0.283]
0.000 0.00 [ 1.905 -0.842]
0.000 0.00 [ 2.339 -0.242]
0.000 0.00 [ 2.389 -0.261]
0.000 0.00 [3.12 0.02]
0.000 0.00 [3.235 0.06 ]
0.000 0.00 [3.207 0.448]
0.000 0.00 [ 3.242 -0.377]
0.000 0.00 [3.526 0.325]
0.000 0.00 [4.016 0.053]
0.000 0.00 [ 4.353 -0.165]


In [183]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(range(len(angle_list[3:])), [x[0] for x in shift_list[3:]])
ax.plot(range(len(angle_list[3:])), [x[1] for x in shift_list[3:]])

plt.show()

In [155]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)


im = ax.imshow(moving_image)
fig.colorbar(im, ax=ax)

plt.show()

In [161]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

index=20

scaler = data_list[index]['i0'].copy()
scaler[scaler < 1000] = np.mean(scaler)
scaler[scaler > 1e6] = np.median(scaler)

moving_image = data_list[index][element] / scaler

im = ax.imshow(moving_image, vmin=0.88*np.max(moving_image), vmax=0.35)
fig.colorbar(im, ax=ax)

plt.show()

In [175]:
save_dir = f'{base}plots//'

element_key = 'Sb_L'

def _plot_image(index):
    scaler = data_list[index]['i0'].copy()
    scaler[scaler < 1000] = np.mean(scaler)
    scaler[scaler > 1e6] = np.median(scaler)

    plot_img = data_list[index][element_key] / scaler
    plot_img = rotate(plot_img, -angle_list[index])
    shift = shift_list[index]
    #print(shift)

    element_img = ax.imshow(plot_img, extent=[0 + shift[1],
                                              ref_image.shape[1] + shift[1],
                                              0 - shift[0],
                                              ref_image.shape[0] - shift[0]])
    
    return element_img

for ind in range(len(data_list)):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

    ax.set_xlim(0, ref_image.shape[1])
    ax.set_ylim(0, ref_image.shape[0])

    _plot_image(ind)

    plt.savefig(f'{save_dir}BNL_inner_aligned_{ind}.png', transparent=True)
    plt.close()

In [174]:
element_key = 'Sb_L'
ref_image = data_list[0][element] / data_list[0]['i0']


fig = plt.figure(figsize=(5, 5), dpi=200)
ax = fig.add_axes([0.1, 0.1, 0.5, 0.8])

def _plot_image(index):
    scaler = data_list[index]['i0'].copy()
    scaler[scaler < 1000] = np.mean(scaler)
    scaler[scaler > 1e6] = np.median(scaler)

    plot_img = data_list[index][element_key] / scaler
    plot_img = rotate(plot_img, -angle_list[index])
    shift = shift_list[index]
    #print(shift)

    element_img = ax.imshow(plot_img, extent=[0 + shift[1],
                                              ref_image.shape[1] + shift[1],
                                              0 - shift[0],
                                              ref_image.shape[0] - shift[0]],
                                              vmin=0.2, vmax=0.35)
    
    return element_img

element_img = _plot_image(0)
#element_img = ax.imshow(data_list[0][element_key] / data_list[0]['i0'])

ax.set_xlim(0, ref_image.shape[1])
ax.set_ylim(0, ref_image.shape[0])

slider_ax = fig.add_axes([0.7, 0.1, 0.03, 0.8])
energy_slider = Slider(
    ax=slider_ax,
    label='Index',
    valmin=0,
    valmax=len(energy_list) - 1,
    valinit=0,
    orientation='vertical'
)

# The function to be called anytime a slider's value changes
def update(val):
    global element_img
    val_ind = int(np.round(val, 0))
    element_img.remove()
    element_img = _plot_image(val_ind)
    #element_img = ax.imshow(data_list[val_ind][element_key] / data_list[val_ind]['i0'])
    fig.canvas.draw_idle()
    
energy_slider.on_changed(update)

plt.show()

In [246]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.imshow(ref_image, extent=[0, ref_image.shape[1], 0, ref_image.shape[0]])

ax.set_xlim(0, ref_image.shape[1])
ax.set_ylim(0, ref_image.shape[0])

plt.show()

In [247]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.imshow(rotate(moving_image, -recovered_angle), extent=[0 + shift[1],
                                ref_image.shape[1] + shift[1],
                                0 - shift[0],
                                ref_image.shape[0] - shift[0]])

ax.set_xlim(0, ref_image.shape[1])
ax.set_ylim(0, ref_image.shape[0])

plt.show()